# Project: Data Lake with Spark

In [ ]:
## Import Modules

In [45]:
import configparser
import os
import pyspark.sql.functions as F

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.window import Window

In [ ]:
## Create Spark Session

In [27]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .getOrCreate()
    return spark

spark = create_spark_session()

## Set AWS credentials

In [62]:
config = configparser.ConfigParser()

config.read_file(open('aws/dl.cfg'))

os.environ["AWS_ACCESS_KEY_ID"]= config['AWS']['AWS_ACCESS_KEY_ID']
os.environ["AWS_SECRET_ACCESS_KEY"]= config['AWS']['AWS_SECRET_ACCESS_KEY']

#### Test AWS S3 Access

## Data Source

In [29]:
# input_data = "s3a://udacity-dend/"

input_data = "data/"
output_data = "output/"

## Process Song data

#### Get filepath to song data file

In [30]:
song_data = input_data + "song_data/*/*/*/"

#### Read song data file

In [31]:
song_df = spark.read.json(song_data)

#### Create songs table

In [32]:
songs_table = song_df.select(["song_id", "title", "artist_id", "year", "duration"]).distinct()
songs_table.show(5, False)

+------------------+---------------------------------------------------+------------------+----+---------+
|song_id           |title                                              |artist_id         |year|duration |
+------------------+---------------------------------------------------+------------------+----+---------+
|SOGOSOV12AF72A285E|¿Dónde va Chichi?                                  |ARGUVEV1187B98BA17|1997|313.12934|
|SOTTDKS12AB018D69B|It Wont Be Christmas                               |ARMBR4Y1187B9990EB|0   |241.47546|
|SOBBUGU12A8C13E95D|Setting Fire to Sleeping Giants                    |ARMAC4T1187FB3FA4C|2004|207.77751|
|SOIAZJW12AB01853F1|Pink World                                         |AR8ZCNI1187B9A069B|1984|269.81832|
|SONYPOM12A8C13B2D7|I Think My Wife Is Running Around On Me (Taco Hell)|ARDNS031187B9924F0|2005|186.48771|
+------------------+---------------------------------------------------+------------------+----+---------+
only showing top 5 rows



#### Create artists table

In [33]:
artists_table = song_df.select(["artist_id", "artist_name", "artist_location", "artist_latitude", "artist_longitude"]).distinct()
artists_table.show(5, truncate = False)

+------------------+---------------+---------------+---------------+----------------+
|artist_id         |artist_name    |artist_location|artist_latitude|artist_longitude|
+------------------+---------------+---------------+---------------+----------------+
|AR3JMC51187B9AE49D|Backstreet Boys|Orlando, FL    |28.53823       |-81.37739       |
|AR0IAWL1187B9A96D0|Danilo Perez   |Panama         |8.4177         |-80.11278       |
|ARWB3G61187FB49404|Steve Morse    |Hamilton, Ohio |null           |null            |
|AR47JEX1187B995D81|SUE THOMPSON   |Nevada, MO     |37.83721       |-94.35868       |
|ARHHO3O1187B989413|Bob Azzam      |               |null           |null            |
+------------------+---------------+---------------+---------------+----------------+
only showing top 5 rows



## Process Log data

#### Get filepath to log data file

In [35]:
log_data = input_data + "log_data/"

#### Read log data file

In [36]:
log_df = spark.read.json(log_data)

#### filter by actions for song plays

In [37]:
log_df = log_df.where('page="NextSong"')
log_df.show(2)

+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|     artist|     auth|firstName|gender|itemInSession|lastName|   length|level|            location|method|    page|     registration|sessionId|           song|status|           ts|           userAgent|userId|
+-----------+---------+---------+------+-------------+--------+---------+-----+--------------------+------+--------+-----------------+---------+---------------+------+-------------+--------------------+------+
|   Harmonia|Logged In|     Ryan|     M|            0|   Smith|655.77751| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|  Sehr kosmisch|   200|1542241826796|"Mozilla/5.0 (X11...|    26|
|The Prodigy|Logged In|     Ryan|     M|            1|   Smith|260.07465| free|San Jose-Sunnyval...|   PUT|NextSong|1.541016707796E12|      583|The Big Gundown|

### Create users table

In [38]:
users_table = log_df.select(["userId", "firstName", "lastName", "gender", "level"]).distinct()
users_table.show(5, truncate = False)

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|57    |Katherine|Gay     |F     |free |
|84    |Shakira  |Hunt    |F     |free |
|22    |Sean     |Wilson  |F     |free |
|52    |Theodore |Smith   |M     |free |
|80    |Tegan    |Levine  |F     |paid |
+------+---------+--------+------+-----+
only showing top 5 rows



### Create time table

#### Create timestamp column from original "ts" column

In [39]:
log_df = log_df.withColumn('timestamp',( (log_df.ts.cast('float')/1000).cast("timestamp")) )
log_df.select('timestamp').show(3, False)

+-----------------------+
|timestamp              |
+-----------------------+
|2018-11-15 00:29:39.712|
|2018-11-15 00:40:35.072|
|2018-11-15 00:44:57.216|
+-----------------------+
only showing top 3 rows



#### Create **time_table**

In [40]:
time_table = log_df.select(
                    F.col("timestamp").alias("start_time"),
                    F.hour("timestamp").alias('hour'),
                    F.dayofmonth("timestamp").alias('day'),
                    F.weekofyear("timestamp").alias('week'),
                    F.month("timestamp").alias('month'), 
                    F.year("timestamp").alias('year'), 
                    F.date_format(F.col("timestamp"), "E").alias("weekday")
                )


time_table.show(5, False)

+-----------------------+----+---+----+-----+----+-------+
|start_time             |hour|day|week|month|year|weekday|
+-----------------------+----+---+----+-----+----+-------+
|2018-11-15 00:29:39.712|0   |15 |46  |11   |2018|Thu    |
|2018-11-15 00:40:35.072|0   |15 |46  |11   |2018|Thu    |
|2018-11-15 00:44:57.216|0   |15 |46  |11   |2018|Thu    |
|2018-11-15 03:44:05.12 |3   |15 |46  |11   |2018|Thu    |
|2018-11-15 05:48:36.224|5   |15 |46  |11   |2018|Thu    |
+-----------------------+----+---+----+-----+----+-------+
only showing top 5 rows



### Create song play table

#### Read Song data file

In [41]:
song_df = spark.read.json(input_data+'song_data/*/*/*/*.json')

#### Join song_df abd log_df

In [42]:
song_log_joined_table = log_df.join(song_df, (log_df.song == song_df.title) & (log_df.artist == song_df.artist_name) & (log_df.length == song_df.duration), how='inner')

#### Extract columns to create songplays table

In [47]:
songplays_table = song_log_joined_table.distinct() \
                    .select("userId", "timestamp", "song_id", "artist_id", "level", "sessionId", "location", "userAgent" ) \
                    .withColumn("songplay_id", F.row_number().over( Window.partitionBy("timestamp").orderBy("timestamp"))) \
                    .withColumnRenamed("userId","user_id")        \
                    .withColumnRenamed("timestamp","start_time")  \
                    .withColumnRenamed("sessionId","session_id")  \
                    .withColumnRenamed("userAgent", "user_agent") \
                    

songplays_table.show(5)

+-------+--------------------+------------------+------------------+-----+----------+--------------------+--------------------+-----------+
|user_id|          start_time|           song_id|         artist_id|level|session_id|            location|          user_agent|songplay_id|
+-------+--------------------+------------------+------------------+-----+----------+--------------------+--------------------+-----------+
|     15|2018-11-21 21:56:...|SOZCTXZ12AB0182364|AR5KOSW1187FB35FF4| paid|       818|Chicago-Napervill...|"Mozilla/5.0 (X11...|          1|
+-------+--------------------+------------------+------------------+-----+----------+--------------------+--------------------+-----------+



## Write to parquet files locally

In [48]:
songs_table.write.parquet(output_data + 'songs/' + 'artists.parquet')
artists_table.write.parquet(output_data + 'artists/' + 'artists.parquet')
users_table.write.parquet(output_data + 'users/' + 'artists.parquet')
time_table.write.parquet(output_data + 'time/' + 'artists.parquet')
songplays_table.write.parquet(output_data + 'songplays/' + 'artists.parquet')

## Create AWS S3 Bucket

In [56]:
import boto3

bucket_root='udacity-de-sparkify-data-lakes'
s3 = boto3.resource('s3', region_name='us-west-2')

In [57]:
try:
        s3.create_bucket(ACL='private',Bucket=bucket_root,
                             CreateBucketConfiguration={'LocationConstraint':'us-west-2'})
except Exception as e:
        if 'BucketAlreadyOwnedByYou' in str(e):
            print(f'{bucket_root} already exists')
        else:
           raise e

In [58]:
bucket = s3.Bucket(bucket_root)
folders_name = ['songs','songplays','time','artists','users']

# Create folders for the tables
for f in folders_name:
    fold_name = f'{f}/'
    try: 
        bucket.put_object(Key=fold_name)
    except Exception as e:
        raise e
print('Folders created')

Folders created


In [59]:
bucket = s3.Bucket(bucket_root  )

for obj in bucket.objects.all():
    print(obj.key)

artists/
songplays/
songs/
time/
users/


## Write to AWS S3 Bucket

In [61]:
output_data  = "s3a://udacity-de-sparkify-data-lakes/" 

#songplays_table.write.parquet(output_data + 'songplays/' + 'songplays.parquet')
songs_table.write.parquet(output_data + 'songs/' + 'songs.parquet')
#artists_table.write.parquet(output_data + 'artists/' + 'artists.parquet')
#users_table.write.parquet(output_data + 'users/' + 'users.parquet')
#time_table.write.parquet(output_data + 'time/' + 'time.parquet')

Py4JJavaError: An error occurred while calling o328.parquet.
: com.amazonaws.AmazonClientException: Unable to load AWS credentials from any provider in the chain
	at com.amazonaws.auth.AWSCredentialsProviderChain.getCredentials(AWSCredentialsProviderChain.java:117)
	at com.amazonaws.services.s3.AmazonS3Client.invoke(AmazonS3Client.java:3521)
	at com.amazonaws.services.s3.AmazonS3Client.headBucket(AmazonS3Client.java:1031)
	at com.amazonaws.services.s3.AmazonS3Client.doesBucketExist(AmazonS3Client.java:994)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.initialize(S3AFileSystem.java:297)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:2669)
	at org.apache.hadoop.fs.FileSystem.access$200(FileSystem.java:94)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:2703)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:2685)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:373)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:295)
	at 